In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("Sleep_health_dataset.csv")
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [6]:
df["Age"].min(), df["Age"].max()

(27, 59)

In [8]:
df["Sleep Duration"].min(), df["Sleep Duration"].max()

(5.8, 8.5)

In [10]:
df["Physical Activity Level"].min(), df["Physical Activity Level"].max()

(30, 90)

In [12]:
df["Stress Level"].min(), df["Stress Level"].max()

(3, 8)

In [14]:
df["Blood Pressure"].unique()

array(['126/83', '125/80', '140/90', '120/80', '132/87', '130/86',
       '117/76', '118/76', '128/85', '131/86', '128/84', '115/75',
       '135/88', '129/84', '130/85', '115/78', '119/77', '121/79',
       '125/82', '135/90', '122/80', '142/92', '140/95', '139/91',
       '118/75'], dtype=object)

In [16]:
df["Heart Rate"].min(), df["Heart Rate"].max()

(65, 86)

In [18]:
df["Daily Steps"].min(), df["Daily Steps"].max()

(3000, 10000)

In [65]:
X_train = pd.read_csv("X_train_to_preprocess.csv")
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   336 non-null    object 
 1   Age                      336 non-null    int64  
 2   Occupation               336 non-null    object 
 3   Sleep Duration           336 non-null    float64
 4   Physical Activity Level  336 non-null    int64  
 5   Stress Level             336 non-null    int64  
 6   BMI Category             336 non-null    object 
 7   BP_up                    336 non-null    int64  
 8   BP_down                  336 non-null    int64  
 9   Heart Rate               336 non-null    int64  
 10  Daily Steps              336 non-null    int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 29.0+ KB


In [9]:
X_train.columns.tolist()

['Gender',
 'Age',
 'Occupation',
 'Sleep Duration',
 'Physical Activity Level',
 'Stress Level',
 'BMI Category',
 'BP_up',
 'BP_down',
 'Heart Rate',
 'Daily Steps']

In [17]:
import pickle as pk

model_path = open("GB_model.pkl",'rb')
model = pk.load(model_path)
model

GradientBoostingClassifier(min_samples_leaf=5, n_estimators=300)

In [23]:
len(model.feature_importances_)

21

In [83]:
pd.DataFrame(X_train.iloc[22]).T

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,BP_up,BP_down,Heart Rate,Daily Steps
22,Male,38,Lawyer,7.3,60,5,Normal,130,85,68,8000


In [265]:
from sklearn.preprocessing import RobustScaler
def feature_scaling(test_data):
    scaler = RobustScaler()
    num_cols = X_train.select_dtypes(exclude="object").columns.tolist()
    scaler.fit(X_train[num_cols])
    test_data[num_cols] = scaler.transform(test_data[num_cols])
    return test_data

In [267]:
train = feature_scaling(test)
train

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,BP_up,BP_down,Heart Rate,Daily Steps
0,Male,-1.357143,Nurse,0.153846,-0.5,0.333333,Obese,-1.0,-0.5,0.75,-1.666667


In [269]:
from sklearn.preprocessing import OneHotEncoder

def feature_encoding(test_data):
    ohe = OneHotEncoder(sparse_output=False, drop="first", handle_unknown='ignore')
    cat_cols = X_train.select_dtypes(include="object").columns.tolist()
    ohe.fit(X_train[cat_cols])
    test_encoded = ohe.transform(test_data[cat_cols])
    test_encoded = pd.DataFrame(test_encoded, columns=ohe.get_feature_names_out(cat_cols)).astype("int64")
    test_data.drop(cat_cols, axis=1, inplace=True)
    test_data = pd.concat([test_encoded, test_data], axis=1)
    return test_encoded, test_data

In [271]:
encoded, train = feature_encoding(train)

In [273]:
train

,Gender_Male,Occupation_Doctor,Occupation_Engineer,Occupation_Lawyer,Occupation_Manager,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,...,BMI Category_Obese,BMI Category_Overweight,Age,Sleep Duration,Physical Activity Level,Stress Level,BP_up,BP_down,Heart Rate,Daily Steps
0,1,0,0,0,0,1,0,0,0,0,...,1,0,-1.357143,0.153846,-0.5,0.333333,-1.0,-0.5,0.75,-1.666667


In [99]:
X_train["BMI Category"].unique().reshape(1,-1)

array([['Normal', 'Overweight', 'Obese']], dtype=object)

In [111]:
data = { "Category" : ["red"] }
pd.DataFrame(data)

,Category
0,red


In [137]:
temp = pd.DataFrame(X_train.iloc[22,:]).T
temp

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,BP_up,BP_down,Heart Rate,Daily Steps
22,Male,38,Lawyer,7.3,60,5,Normal,130,85,68,8000


In [167]:
temp_cols = temp.select_dtypes(include='object').columns.tolist()
temp[temp_cols]

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,BP_up,BP_down,Heart Rate,Daily Steps
22,Male,38,Lawyer,7.3,60,5,Normal,130,85,68,8000


In [165]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 22 to 22
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Gender                   1 non-null      object
 1   Age                      1 non-null      object
 2   Occupation               1 non-null      object
 3   Sleep Duration           1 non-null      object
 4   Physical Activity Level  1 non-null      object
 5   Stress Level             1 non-null      object
 6   BMI Category             1 non-null      object
 7   BP_up                    1 non-null      object
 8   BP_down                  1 non-null      object
 9   Heart Rate               1 non-null      object
 10  Daily Steps              1 non-null      object
dtypes: object(11)
memory usage: 96.0+ bytes


In [213]:
X_train.dtypes

Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
BP_up                        int64
BP_down                      int64
Heart Rate                   int64
Daily Steps                  int64
dtype: object

In [257]:
data_to_predict = [str("Male"), np.int64(24), str("Nurse"), np.float64(7.4), np.int64(45), np.int64(6), str("Obese"), np.int64(120), np.int64(80), np.int64(73), np.int64(3500)]
test = pd.DataFrame(data=np.array(data_to_predict).reshape(1,-1),
                        columns=X_train.columns.tolist())
test

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,BP_up,BP_down,Heart Rate,Daily Steps
0,Male,24,Nurse,7.4,45,6,Obese,120,80,73,3500


In [259]:
for i in X_train.columns:
    test[i] = test[i].astype(X_train.dtypes[i])

In [261]:
test.dtypes

Gender                      object
Age                          int64
Occupation                  object
Sleep Duration             float64
Physical Activity Level      int64
Stress Level                 int64
BMI Category                object
BP_up                        int64
BP_down                      int64
Heart Rate                   int64
Daily Steps                  int64
dtype: object

In [263]:
test

,Gender,Age,Occupation,Sleep Duration,Physical Activity Level,Stress Level,BMI Category,BP_up,BP_down,Heart Rate,Daily Steps
0,Male,24,Nurse,7.4,45,6,Obese,120,80,73,3500
